# What's this code about
Interface LAMMPS / HPC and do job controls on jupyter notebook. Use this code to submit jobs. if goal is submitting many jobs, please just create folders and use job array (good practice),
or use csv to tabulate the parameters used and iterate over them in a script

# How to use
1. prepare folders containing the desired simulation parameters/settings and force field in your lammps script "in.hybrid" and your HPC slurm script "equil.sh". Specify the location of the scripts "mother_folder" and
your designated folder to save simulation file as variable "save_path" (see TODO in this notebook).
2. Run this notebook in HPC. It is also possible to do it locally (contact Po-An)
## Future direction
maybe incorporate this with the package Signac

In [ ]:
import freud
import warnings
import numpy as np
import shutil
import os.path
import time
import glob
def Patch_Position_Writing(p):
  '''
  Function writes position (coordinate) of patch (if they exist on the design what would the position be) on a square. Input p (p as protrusion ratio).
  
  '''

  arr1_table = []
  arr2_table = []
  arr3_table = []
  arr4_table = []
  for i in range(3):  #particle size 3*3
    x1 = -1 +i #Draw a dummy figure then you'll know what's going on with these two for loops
    y1 = 1 + p
    x3 = 1 - i #not -1+i. change here if generalizing to other scripts. contact poan for the general scripts for patchy polygons
    y3 = -1 - p
    arr1_table.append([x1,y1])
    arr3_table.append([x3,y3])

  for j in range(3):
    x2 = 1 + p
    y2 = 1 - j
    x4 = -1 - p
    y4 = -1 + j 
    arr2_table.append([x2,y2])
    arr4_table.append([x4,y4])


  arr1_table = np.array(arr1_table)
  arr2_table = np.array(arr2_table)
  arr3_table = np.array(arr3_table)
  arr4_table = np.array(arr4_table)
  return np.array([arr1_table,arr2_table, arr3_table,arr4_table])

def read_input(file_folder):
    '''
    Legacy / obsolete function, ignore this. Read file 'design.txt' in each folder and generate training data
    '''
    file_pos = "{}/design.txt".format(file_folder)
    _x_train = []
    with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            protrusion = np.genfromtxt(file_pos, skip_header=1, max_rows=1)
            data = np.genfromtxt(file_pos, skip_header=3, invalid_raise=False) #troubles with ragged arrary
            data5 = np.genfromtxt(file_pos, skip_header=7, invalid_raise=False)

    #create training data with features
    _x_train.append(protrusion)
    for arr in data:
        _x_train.append(_map_bin2int_np_dot(arr))#map binary array to integer array
    _x_train.append(_map_bin2int_np_dot(data5))#map binary array to integer array
    return np.array(_x_train)

def write_input(binary_arr1,binary_arr2,binary_arr3,binary_arr4,p, save_path, concentration = 0.00263):
  '''
  Input binary_arr (np array) represent the design, and p means protrusion ratio of particles
  Output lammps script
  
  Note:
  
  concentration (1/sigma^2). in this project concentration = 0.00263 == 50 nM
  
  '''

  length = 1/concentration**0.5 # length of box, 2D in this case
  patch_table = Patch_Position_Writing(p) #Obtain patch location. The patch location is specified in advance and data is retrieved from this specified table.
  binar  = np.array([binary_arr1,binary_arr2,binary_arr3,binary_arr4])
  position = []
  for i in range(4):
    coord = patch_table[i][np.nonzero(binar[i])]
    if coord.size !=0 and coord.ndim !=3:
      position.append(coord)
    elif coord.ndim ==3:
      coord = coord[:,:,0]
      position.append(coord)
        
  #print(position)
  position = np.array(position)
  position = np.concatenate(position)# flatten an array of array to an array

  n = len(position) #Number of patches
  natom = n+9 #totnal number of atoms
  lammps_input = "{}/lammps.input".format(save_path) #save to a specific folder
  o = open(lammps_input, 'w')
  o.write("LAMMPS patchy particle input data file\n\n")
  o.write(str(natom)+" atoms\n")

  o.write("3 atom types\n\n")
  o.write(str(-length/2)+' '+str(length/2)+ " xlo xhi\n")
  o.write(str(-length/2)+' '+str(length/2)+ " ylo yhi\n")
  o.write(str(-0.5)+' '+str(0.5)+ " zlo zhi\n\n")
  bulk = '''
  Masses

  1  1
  2  1
  3  1

  Atoms

  1   1     1      -1.000000      1.000000       0.000000
  2   1     1      0.000000      1.000000       0.000000
  3   1     1      1.000000      1.000000       0.000000
  4   1     1      -1.000000      0.000000       0.000000
  5   1     3      0.000000      0.000000       0.000000
  6   1     1      1.000000      0.000000       0.000000
  7   1     1      -1.000000      -1.000000       0.000000
  8   1     1      0.000000      -1.000000       0.000000
  9   1     1      1.000000      -1.000000       0.000000
  '''
  o.writelines(bulk)

  for i in range(n):
    o.write(str(10+i)+'   1     2      '+str(float(position[i][0]))+' '+ str(float(position[i][1]))+" 0.000000\n")
  o.close()
  design_file = "{}/design.txt".format(save_path) #honestly we don't need this. this line outputs an additional file specifying design name, but we can read design from folder name lol
  o = open(design_file, 'w')
  o.write("Protrusion\n")
  o.write(str(p)+"\n")
  o.write("Binary design 1 to 5\n")
  for arr in binar:
        
        for i in range(len(arr)):
          o.write(str(arr[i])+" ")
        
        o.write("\n")
  o.close()
def _map_bin2int_np_dot(x):
  '''
  Function transforms binary array into an integer. 
  '''
  l = x.size# from question http://stackoverflow.com/questions/41069825/convert-binary-01-numpy-to-integer-or-binary-string
  return np.dot(x,1 << np.arange(l))

#Convert integer to binary
def _map_int2bin(x,digit):
  '''
  Function transforms int into binary array. 
  '''
  _bin = bin(x)[2:].zfill(digit)
  return np.array(list(_bin), dtype=int)

def copy(src, dst):
  '''
  Copy file without triggering permission error

  '''
  if os.path.isdir(dst):
        dst = os.path.join(dst, os.path.basename(src))
  shutil.copyfile(src, dst)

def sleep2read(file_path):
    '''
    Ignore this. Function is useful when multiprocessing is needed. Function stop the process for an hour to let simulation complete.
    '''
    time.sleep(300)# usally takes about an hour to finish.  #now it's like 5 min
    while not os.path.exists(file_path):
        time.sleep(60)#sleep for a minute and check again   

    if os.path.isfile(file_path):
        pass
    else:
        raise ValueError("%s isn't a file!" % file_path)
    return 

def fail_safe(file_path):
    '''
    Ignore this. For multiprocessing. sometimes the process ends early or something. make sure process continues
    '''
    clock= 0
    while not os.path.exists(file_path):
        time.sleep(1)#sleep for a second and check again
        clock+=1
        
        if clock >=400:
            print('sth wrong with {}'.format(file_path))

    return 

def perm(num):
  '''
  get 4 digits number in string format, return 3 possible numbers by putting the end digit to the top
  '''
  num_st = str(num)
  t1 = num_st[3]+num_st[:3]
  t2 = t1[3]+t1[:3]
  t3 = t2[3]+t2[:3]
  return t1,t2,t3

def check_for_files(filepath):
    '''
    I use this function to check whether an error exist (core dump file exist)
    '''
    for filepath_object in glob.glob(filepath):
        if os.path.isfile(filepath_object):
            return True

    return False

In [ ]:

def sim_job_submit(binary_arr1,binary_arr2,binary_arr3,binary_arr4,p,s_dir,concentration = 0.00263, mother_folder = "lammps_script"):    
    '''
    submit LAMMPS (equilibration simulation) jobs in Expanse supercomputer.
    Input: binary_arr1,binary_arr2,binary_arr3,binary_arr4 are design specification (patch on an edge) like [1,0,1]. Four edges so four array. p is protrusion ratio, s_dir is the saving directory of that design.
            concentration is the number density of patchy particles. mother_folder contains the lammps script

    Outcome: maake folders with simulations script, submit jobs on HPC
    '''
    check_dummy = '{}/restart_equil.equil'.format(s_dir) #If this file exist, skip the submitting job
    #mother_folder = '/expanse/lustre/projects/ddp381/pl201/nanorod/nanocube/2D/dataset_build/royal_simulation'
    #mother_folder = '/expanse/lustre/projects/ddp381/pl201/nanorod/nanocube/2D/dataset_build/royal_simulation/fixed_0_5_volume_frac'
    if not os.path.exists(check_dummy): #Check if file exist
        if not os.path.exists(s_dir): #Check if folder exist
            !mkdir {s_dir}
        else:
            print("something wrong with {}".format(s_dir))
        #write_input(binary_arr1,binary_arr2,binary_arr3,binary_arr4,p,s_dir)
        write_input(binary_arr1,binary_arr2,binary_arr3,binary_arr4,p,s_dir, concentration = concentration)
        !cp {mother_folder}/in.hybrid {mother_folder}/equil_submit.sh {s_dir}
        !sbatch --chdir={s_dir} {s_dir}/equil_submit.sh

    else:
        pass
    
    
def sim_flow2dir(x,p,save_path= 'sim_data'):
    """input design, create corresponding folder containing input script and trajectories of simulaiton data.
    x has form like '1104', p is float. first to four elements represent design. Last element represent protrusion
    d1~d4 has integer value
    The function is concept is based on the flow to directory
    """
 
    #Construct our parameters:
    d1,d2,d3,d4 = x[0],x[1],x[2],x[3]

    binary_arr1 = _map_int2bin(int(d1),3) #round the float to int because only integer value can be turned to binary 
    binary_arr2 = _map_int2bin(int(d2),3) #round the float to int
    binary_arr3 = _map_int2bin(int(d3),3) #round the float to int
    binary_arr4 = _map_int2bin(int(d4),3) #round the float to int
    idx = 10000*int(d1)+1000*int(d2)+100*int(d3)+10*int(d4)+p#generate unique index for simulation, so that I can find them later

    #s_dir = '/expanse/lustre/projects/ddp381/pl201/nanorod/nanocube/2D/dataset_build/royal_data/origin_conc/{0:.2f}'.format(idx)
    #s_dir = '/expanse/lustre/projects/ddp381/pl201/nanorod/nanocube/2D/dataset_build/royal_data/fixed_0_5_volume/{0:.2f}'.format(idx) #write input for lammps, s_dir stands for simulation directory, specify to two digits
    s_dir = '{}/{0:.2f}'.format(save_path,idx)
    return binary_arr1,binary_arr2,binary_arr3,binary_arr4,p,s_dir
    
    #fail_safe(s_dir)#For multiprocessing
def job_submit_prod(s_dir):    
    '''
     submit LAMMPS (production simulation) jobs in Expanse supercomputer
    '''
    if not os.path.exists('{}/cool_1.xyz'.format(s_dir)): #Check if production has been run. In my case, after production run there should be cool_1.xyz in the folder
        !sbatch --chdir={s_dir} {s_dir}/product_submit.sh
    else:
            pass

def garbage_finder(s_dir):    
    '''
     find which folder is being naughty (didn't do production). Sometimes LAMMPS crash for no reason especially 2D simulation. There's a existing bug (see z coordinate drift discussion on lammps forum)
    '''
    check_dummy = '{}/restart_equil_5.equil'.format(s_dir) #IF this file exist, skip the submitting job
    if not os.path.exists(check_dummy): #Check if file exist
        if not os.path.exists(s_dir): #Check if folder exist
            !mkdir {s_dir}
        else:
            print("something wrong with {}".format(s_dir))
    else:
        pass
    
def prod_garbage_finder(s_dir):    
    '''
     find which folder is being naughty (didn't do production)
    '''
    
    filelist = ['cool_1.xyz','cool_2.xyz','cool_3.xyz','cool_4.xyz','cool_5.xyz']


    cool_list = []

    for file in filelist:
            check_dummy = '{}/{}'.format(s_dir,file) #IF this file exist, skip the submitting job
            cool_list.append(os.path.isfile(check_dummy))

    if all(cool_list):
            # All elements are True. Therefore all the files exist. Run %run commands
            pass
    else:
            # At least one element is False. Therefore not all the files exist. Run FTP commands again
            print("something wrong with {}".format(s_dir)) # wait 10 minutes before checking again
            
  
    
def garbage_fixer(binary_arr1,binary_arr2,binary_arr3,binary_arr4,p,s_dir):    
    '''
     submit LAMMPS (equilibration simulation) jobs in Expanse supercomputer with finer timestep
    '''
    check_dummy = '{}/restart_equil_5.equil'.format(s_dir) #IF this file exist, skip the submitting job
    if not os.path.exists(check_dummy): #Check if file exist
        if not os.path.exists(s_dir): #Check if folder exist
            !mkdir {s_dir}
            !cp in.hybrid {s_dir}
        else:
            print("something wrong with {}".format(s_dir))
        write_input(binary_arr1,binary_arr2,binary_arr3,binary_arr4,p,s_dir)
        !cp finer_equilibration/in.hybrid product.hybrid finer_equilibration/equil_submit.sh product_submit.sh {s_dir}
        !sbatch --chdir={s_dir} {s_dir}/equil_submit.sh
        #if not os.path.exists('{}/cool.xyz'.format(s_dir)): #check if cool.xyz file exist
        #    !sbatch --chdir={s_dir} {s_dir}/submit_BO.sh  #change working directories
        #else:
        #    pass
    else:
        pass


In [ ]:
# This entire section of codes gets the total possible unique combination of design
unique_rep = np.array([])
for i in range(7777): #int 1 to 7778
  s1 = 7777-i #string1

  s1 = str(s1).zfill(4) #pad to 4 digits 11 -> 0011
  if '8' in s1: #check if 8 9 is in the digit (8 9 not allowed, the max is 7)
    #print(s1)
    pass
  elif "9" in s1:
    pass
  else:
       
    #append zero if needed
    s2, s3, s4 = perm(s1)
    perm_arr = np.array ( [s1, s2, s3, s4 ]) #make them np array for easy operation
    duplicate = []
    #between group testing -test with the current unique rep
    for ele in perm_arr:
      duplicate_test = ele==unique_rep
      if np.any(duplicate_test): #if there is any duplicates, do nothing
        duplicate.append(True)
        pass

      else:
        duplicate.append(False)

    if np.any(duplicate): #if there is any duplicates, do nothing
      pass
    else:
      unique_rep = np.append(unique_rep,s1) #append the unique


In [ ]:
print("We have {} unique representation".format(unique_rep.size))

1043 unique representations x 10 protrusion ratios = 10430. This is the total possible design

In [ ]:
#Run this cell cautiously
#create data

mother_folder = TODO #spcify the LAMMPS script location. Contact PoAn for LAMMPS script
save_path = TODO #where to save the simulation data. it will be huge so be prepared...

high_conc = 0.5/16
protrusion_grid = np.linspace(0.1,1.0,10) # array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])
for design_i in unique_rep: #1100 for instance
    for protrusion_j in protrusion_grid:
        binary_arr1,binary_arr2,binary_arr3,binary_arr4,p,s_dir = sim_flow2dir(design_i,protrusion_j, save_path =save_path)
        sim_job_submit(binary_arr1,binary_arr2,binary_arr3,binary_arr4,p,s_dir, high_conc, mother_folder =mother_folder)